In [38]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

In [39]:
# Import our input dataset
corporate_giving_df = pd.read_csv("charity_data.csv")
corporate_giving_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


## Preprocessing

### Inspect Data

In [40]:
# if len(corporate_giving_df[corporate_giving_df.duplicated(['EIN'])]) == 0:
#     print(f"There are {len(corporate_giving_df[corporate_giving_df.duplicated(['EIN'])])} duplicate EINs; use EIN as index")
# else:
#     print("Do not use EIN as index")

In [41]:
# corporate_giving_df = corporate_giving_df.set_index('EIN')
# corporate_giving_df

In [42]:
corporate_giving_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [43]:
corporate_giving_df.nunique(axis=0)

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

### Drop non-informative columns

In [44]:
# corporate_giving_df = corporate_giving_df.drop(['NAME'], axis=1)
corporate_giving_df = corporate_giving_df.drop(['EIN','NAME'], axis=1)

### Bin categorical columns

In [46]:
corporate_giving_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [47]:
# Binning application type
corporate_giving_df = corporate_giving_df.copy()

app_counts = corporate_giving_df['APPLICATION_TYPE'].value_counts()

replace_app = list(app_counts[app_counts < 500].index)

for app in replace_app:
    corporate_giving_df['APPLICATION_TYPE'] = corporate_giving_df['APPLICATION_TYPE'].replace(app,"Other")
    
corporate_giving_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [48]:
corporate_giving_df['CLASSIFICATION'].value_counts().head(35)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
Name: CLASSIFICATION, dtype: int64

In [49]:
# Binning classification
class_counts = corporate_giving_df['CLASSIFICATION'].value_counts()

replace_class = list(class_counts[class_counts < 700].index)

for class_ in replace_class:
    corporate_giving_df['CLASSIFICATION'] = corporate_giving_df['CLASSIFICATION'].replace(class_,"Other")
    
corporate_giving_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [50]:
corporate_giving_df['SPECIAL_CONSIDERATIONS'] = corporate_giving_df['SPECIAL_CONSIDERATIONS'].replace(to_replace=['N', 'Y'], value=[0, 1])
corporate_giving_df.astype({'SPECIAL_CONSIDERATIONS':'int64'})
corporate_giving_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1


In [51]:
corporate_giving_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,0,5000,1


### Encode categorical columns

In [52]:
# Generate categorical variable list
corporate_giving_cat = corporate_giving_df.dtypes[corporate_giving_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
enc_df = pd.DataFrame(enc.fit_transform(corporate_giving_df[corporate_giving_cat]))

# Add the encoded variable names to the DataFrame
enc_df.columns = enc.get_feature_names(corporate_giving_cat)
enc_df.head()


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [53]:
corporate_giving_df.reset_index()

,index,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1
1,1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1
2,2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
3,3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1
4,4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,34294,T4,Independent,C1000,ProductDev,Association,1,0,0,5000,0
34295,34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
34296,34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,0,5000,0
34297,34297,T5,Independent,C3000,ProductDev,Association,1,0,0,5000,1


In [ ]:
# Merge one-hot encoded features and drop originals
corporate_giving_df = corporate_giving_df.merge(enc_df,left_index=True, right_index=True)
corporate_giving_df = corporate_giving_df.drop(corporate_giving_cat,1)
corporate_giving_df

### Split preprocessed data into training and testing sets and scale

In [55]:
# Split our preprocessed data into our features and target arrays
y = corporate_giving_df["IS_SUCCESSFUL"].values
X = corporate_giving_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)